<a href="https://colab.research.google.com/github/tristantoupin/COMP551-KaggleCompetition/blob/master/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
path_to_data = "../data/train_images.npy"
path_to_labels = "../data/train_labels.csv"
path_to_test = "../data/test_images.npy"

In [5]:
def get_data(path_x, path_y):
    data = np.load(path_x, encoding = 'bytes')
    labels_df = pd.read_csv(path_y)
    labels_df.Category = pd.Categorical(labels_df.Category)
    y = labels_df.Category.cat.codes.values
    X = list(data[:, 1])
    return X, y

In [ ]:
X_train, y_train = get_data(path_to_data, path_to_labels)
print("Shape of training dataset:", len(X_train))
X_test = list(np.load(path_to_test, encoding = 'bytes')[:, 1])
print("Shape of testing dataset:", len(X_test))
unique_classes = set(y_train)
print("Number of classes:", len(unique_classes))

In [7]:
x_tr, x_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)

In [8]:
# parameters
learning_rates = [1e-4, 1e-3, 1e-2, 1e-1]
nodes = [50, 100, 1000, 10000, 12000]
hidden_layers = [1, 2, 3, 4]
batch_size = [1/100, 1/10, 1]
activation = ['sigmoid', 'tanh', 'relu']

In [61]:
X = np.array([[0, 0, 1], [1, 1, 1], [1, 0, 1], [0, 1, 1]])
y = np.array([[0, 1, 1, 0]]).T
unique_classes = (np.unique(y))

In [62]:
class NN():
    def __init__(self, learning_rate, layers, nodes_per_layer, b_size, activation, random_state = 1234):
        np.random.seed(random_state)

        n = [nodes_per_layer for i in range(layers)] + [len(unique_classes)]
        print(n)
        # init params
        self.weights_0 = 2 * np.random.random(tuple(n[:2])) - 1
        self.weights_1 = 2 * np.random.random(tuple(n[1:3])) - 1
        self.weights_2 = 2 * np.random.random(tuple(n[2:])) - 1
        self.batch_size = b_size
        self.activation_fct = activation
        self.learning_rate = learning_rate


    def sigmoid(self, x, deriv = False):
        if(deriv==True):
            return x*(1-x)
        return 1/(1+np.exp(-x))
    

    def train(self, xs, ys):
        for i in range(5000):
            # Predict
            l0 = xs
            l1 = self.sigmoid(np.dot(l0, self.weights_0))
            l2 = self.sigmoid(np.dot(l1, self.weights_1))
            l3 = self.sigmoid(np.dot(l2, self.weights_2))
            
            # Calculate the error            
            l3_error = ys - l3
            l3_delta = l3_error * self.sigmoid(l3, deriv = True)
            
            l2_error = l3_delta.dot(self.weights_2.T)
            l2_delta = l2_error * self.sigmoid(l2, deriv=True)

            l1_error = l2_delta.dot(self.weights_1.T)
            l1_delta = l1_error * self.sigmoid(l1, deriv=True)
    
            
            self.weights_2 += l2.T.dot(l3_delta) * self.learning_rate
            self.weights_1 += l1.T.dot(l2_delta) * self.learning_rate
            self.weights_0 += l0.T.dot(l1_delta) * self.learning_rate
                

            if not(i % 1000):
                print(i)
                print(np.mean(np.abs(l3_error)))




In [65]:
nn = NN(1, 3, 3, 256, 'sigmoid')

[3, 3, 3, 2]


In [66]:
nn.train(X, y)

0
0.50191447186
1000
0.0155112766056
2000
0.0102718021156
3000
0.00818253691336
4000
0.00699159186549
